# This notebook is for testing predictions and submitting models/results for the competition.

In [1]:
# Installing EfficientNet model without internet due to competition requisites

import os
import sys

sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

import torch
os.listdir("../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master")


from efficientnet_pytorch import EfficientNet


#!pip install efficientnet_pytorch torchtoolbox


In [2]:
from efficientnet_pytorch import EfficientNet
# Imports here
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import pandas as pd
import os
import random
import math
import skimage.io
#from csv_loader import load_csv

# Tiff visualisation imports and downloads
import numpy as np
import tifffile as tiff

# For re-importing python modules
import importlib
import imagecodecs

In [3]:
#use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# Load the image manipulation functions to later use them in the dataloader function.

In [4]:
# first version of dataloader to make predictions with available data from train_images.
# There will be a second version of the loader that will perform image manipulations for running test/submission data
class load_csv(Dataset):
    def __init__(self, csv_file, root_dir, transform=True):
        self.annotations = pd.read_csv(csv_file)# todo remove sample for debug
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
        
    
    def __getitem__(self, index):
        #img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image_id = self.annotations.iloc[index, 0]
        img_path = os.path.join(self.root_dir, str(image_id) +".png")
        image = torch.from_numpy(skimage.io.imread(img_path)).permute(2,0,1).float()
        
        #Image.MAX_IMAGE_PIXELS = None
                
        #image.transform = transforms.RandomResizedCrop(224)
        
        y_label = torch.tensor(int(self.annotations.iloc[index,:]['isup_grade']))
        #isup_grade = int(self.annotations.iloc[index,:]['isup_grade'])
        
        #label = np.zeros(6).astype(np.float32)
        #y_label = label[isup_grade] = 1.
        #y_label = torch.tensor(y_label)
        
        self.transform= transforms.Compose([transforms.ToPILImage(),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
        if self.transform:
            image = self.transform(image)
        
        return image, y_label, image_id

# Define model and validate function

In [5]:
# Define/load base model
model = EfficientNet.from_name('efficientnet-b4')
model.load_state_dict(torch.load("../input/efficientnet-pytorch/efficientnet-b4-e116e8b3.pth"))


model._fc = nn.Sequential(nn.Linear(model._fc.in_features, 216),
                          nn.ReLU(),
                          nn.Linear(216, 36, bias=True),
                          nn.ReLU(),
                          nn.Linear(36, 6, bias=True),
                          nn.LogSoftmax(dim=1))


# Load checkpoint
checkpoint = torch.load("../input/training-model-with-k-folds/model_fold_0.pth")

# Load model parameters
model.load_state_dict(checkpoint['model_state_dict'])
model._fc.load_state_dict(checkpoint['classifier_state_dict'])

# Load other model related components
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
fold = checkpoint['fold']
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [6]:
# Validation function
def validate_data_function(model, test_loader, criterion):
    test_loss = 0
    accuracy = 0
    
    df_test = {'image_id': [],
             'label': [],
             'y_pred': []}
    
    for ii, (inputs, labels, image_id) in enumerate(test_loader):
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        pred_y = output.argmax(dim=1)
        
        
        #ps = torch.exp(output)
        #equality = (labels.argmax(dim=1) == output.argmax(dim=1))
        equality = (labels == output.argmax(dim=1))
        accuracy += equality.type(torch.FloatTensor)
        
        df_test['image_id'].append(image_id[0])
        df_test['label'].append(int(labels[0]))
        df_test['y_pred'].append(int(pred_y[0]))
        
    
    accuracy = accuracy/len(test_loader)
    accuracy.item
    return test_loss, accuracy[0], image_id, pred_y, labels, df_test


In [ ]:
# Analyzing each fold's prediction strength and assigning weights to calculate final k-folds weighted averages for future predictions

In [41]:
# Seeing how well each of the folds predict their own test sets
num_folds = 5
sample_size = 100

pred_y_collection = []
for i in range(num_folds):
    # Load checkpoint
    checkpoint_name = "../input/training-model-with-k-folds/model_fold_" + str(i) + ".pth"
    checkpoint = torch.load(checkpoint_name)

    # Load model parameters
    model.load_state_dict(checkpoint['model_state_dict'])
    model._fc.load_state_dict(checkpoint['classifier_state_dict'])

    # Load other model related components
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    fold = checkpoint['fold']
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    # Load appropriate data (can't use non-test images for testing/predicting)
    file_path = '../input/training-model-with-k-folds/test_fold_' + str(0) + '.csv'
    fold_test_set = pd.read_csv(file_path).copy()
    fold_test_set = fold_test_set.sample(sample_size)
    
    fold_sample_name = 'sample_fold_' + str(i) + '.csv'
    fold_test_set.to_csv(fold_sample_name, sep=",", index=False)
    test_data = load_csv(csv_file=fold_sample_name, root_dir='../input/prostate-cancer-tiles-4x4x128px-downsampling-4x/train_128x4x4_res1/train_128x4x4_res1')
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)
    
    # Run predictions for each fold and hopefully get insights
    model.eval()
    
    with torch.no_grad():
        test_loss, accuracy, image_id, pred_y, labels, df_test = validate_data_function(model, test_loader, criterion)
    pred_y_collection.append(df_test)
    print("|Fold: {}  |Test Accuracy: {:.2f}%".format(fold, accuracy))#*100/len(train_loader)))

|Fold: 0  |Test Accuracy: 0.55%
|Fold: 1  |Test Accuracy: 0.58%
|Fold: 2  |Test Accuracy: 0.52%
|Fold: 3  |Test Accuracy: 0.59%
|Fold: 4  |Test Accuracy: 0.49%


In [42]:
def fold_strengths(df_test, scores=6):
    weights = []
    for i in range(scores):
        df_label = df_test[df_test['label']==i]
        equality = df_label[df_label['y_pred']==i]
        label_accuracy = len(equality)/len(df_label)
        weights.append(label_accuracy)
    total_equality = df_test[df_test['label']==df_test['y_pred']]
    total_accuracy = len(total_equality)/len(df_test)
    return total_accuracy, weights

In [56]:
weights_dic

[[0.8214285714285714,
  0.7575757575757576,
  0.4444444444444444,
  0.14285714285714285,
  0.0,
  0.2222222222222222],
 [0.8461538461538461,
  0.6071428571428571,
  0.375,
  0.23076923076923078,
  0.08333333333333333,
  0.9230769230769231],
 [0.5625,
  0.8695652173913043,
  0.14285714285714285,
  0.4166666666666667,
  0.0,
  0.5384615384615384],
 [0.8709677419354839,
  0.41935483870967744,
  0.4,
  0.25,
  0.14285714285714285,
  0.9230769230769231],
 [0.8064516129032258,
  0.8181818181818182,
  0.14285714285714285,
  0.08333333333333333,
  0.08333333333333333,
  0.2222222222222222]]

In [54]:
weights_dic = []
for i in range(num_folds):
    print("Fold {}".format(i))
    total_accuracy, weights = fold_strengths(pd.DataFrame(pred_y_collection[i]), scores=6)
    weights_dic.append(weights)
    print(weights)
pd.DataFrame(weights_dic).to_csv('weights_dic.csv', sep=",", index=False)

Fold 0
[0.8214285714285714, 0.7575757575757576, 0.4444444444444444, 0.14285714285714285, 0.0, 0.2222222222222222]
Fold 1
[0.8461538461538461, 0.6071428571428571, 0.375, 0.23076923076923078, 0.08333333333333333, 0.9230769230769231]
Fold 2
[0.5625, 0.8695652173913043, 0.14285714285714285, 0.4166666666666667, 0.0, 0.5384615384615384]
Fold 3
[0.8709677419354839, 0.41935483870967744, 0.4, 0.25, 0.14285714285714285, 0.9230769230769231]
Fold 4
[0.8064516129032258, 0.8181818181818182, 0.14285714285714285, 0.08333333333333333, 0.08333333333333333, 0.2222222222222222]


### We see from the above that each model/fold has better prediction skills for certain ISUP scores than others.
### Let's: 
### 1) define random sample of data. Will include training data unavoidably. Perform predictions for each fold model
### 2) see accuracy of simple average of k_fold predictions
### 3) see accuracy of weighed average of k_fold predictions

In [44]:
# 1) defining random rample
test_sample_size = sample_size
sample_test_set = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv').copy()
sample_test_set= sample_test_set.sample(test_sample_size)
sample_test_set.to_csv('sample.csv', sep=",", index=False)


In [57]:
# 1) performing predictions
sample_test_data = load_csv(csv_file='sample.csv', root_dir='../input/prostate-cancer-tiles-4x4x128px-downsampling-4x/train_128x4x4_res1/train_128x4x4_res1')
sample_test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)
    

sample_test_data_preds = []
for i in range(num_folds):
    # Load checkpoint
    checkpoint_name = "../input/training-model-with-k-folds/model_fold_" + str(i) + ".pth"
    checkpoint = torch.load(checkpoint_name)

    # Load model parameters
    model.load_state_dict(checkpoint['model_state_dict'])
    model._fc.load_state_dict(checkpoint['classifier_state_dict'])

    # Load other model related components
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    fold = checkpoint['fold']
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    # Load appropriate data (can't use non-test images for testing/predicting)
    file_path = '../input/training-model-with-k-folds/test_fold_' + str(0) + '.csv'
    fold_test_set = pd.read_csv(file_path).copy()
    fold_test_set = fold_test_set.sample(sample_size)
    
    fold_sample_name = 'sample_fold_' + str(i) + '.csv'
    fold_test_set.to_csv(fold_sample_name, sep=",", index=False)
    test_data = load_csv(csv_file=fold_sample_name, root_dir='../input/prostate-cancer-tiles-4x4x128px-downsampling-4x/train_128x4x4_res1/train_128x4x4_res1')
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)
    
    # Run predictions for each fold and hopefully get insights
    model.eval()
    
    with torch.no_grad():
        test_loss, accuracy, image_id, pred_y, labels, df_test = validate_data_function(model, sample_test_loader, criterion)
    sample_test_data_preds.append(df_test)
    print("|Fold: {}  |Test Accuracy: {:.2f}%".format(fold, accuracy))

|Fold: 0  |Test Accuracy: 0.38%
|Fold: 1  |Test Accuracy: 0.48%
|Fold: 2  |Test Accuracy: 0.53%
|Fold: 3  |Test Accuracy: 0.43%
|Fold: 4  |Test Accuracy: 0.49%


In [59]:
for i in range(num_folds):
    fold_preds = pd.DataFrame(sample_test_data_preds[i])
    name = 'fold_' + str(i) + '_y_pred'
    sample_test_set[name]= [x for x in fold_preds['y_pred']]

In [71]:
sample_test_set

,image_id,data_provider,isup_grade,gleason_score,fold_0_y_pred,fold_1_y_pred,fold_2_y_pred,fold_3_y_pred,fold_4_y_pred,fold_avg,w_score_int
5384,84da53ac6f8828a494a0a8c52a6cd158,radboud,4,4+4,0,0,0,0,0,0,0
3554,58d694da01f1b0a9c828857ada4ce80c,karolinska,0,0+0,1,2,1,2,1,1,1
146,03ca7f8d8c993d3959daf10ef57f4489,karolinska,1,3+3,0,5,5,5,5,4,4
7439,b4543b24750d5bec27fb4918d0a4d543,karolinska,1,3+3,1,1,1,1,1,1,1
10549,fe8e6d622955498e6d221940363f57b4,karolinska,0,0+0,2,1,1,3,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...
1101,1c36c6b2d051fd1952232efb2613808d,radboud,4,4+4,1,4,0,0,0,1,0
8599,cf77075f910326e17c57d890d6bbd6e1,karolinska,3,4+3,0,4,1,1,0,1,1
8067,c318254a6de21ceee63f97a015e90d3d,karolinska,1,3+3,0,0,0,1,1,0,0
9792,ed9a38a9f1dfa898e536cd91f13d98b1,radboud,3,4+3,3,5,3,3,2,3,4


In [70]:
# 2) Get simple average
sample_test_set['fold_avg'] = sample_test_set[['fold_0_y_pred', 'fold_1_y_pred', 'fold_2_y_pred', 'fold_3_y_pred', 'fold_4_y_pred']].mean(axis=1).round(0).astype(int) 

equality = sample_test_set[sample_test_set['isup_grade']==sample_test_set['fold_avg']]
accuracy = len(equality)/len(sample_test_set)
accuracy

0.11

In [53]:
weights_dict

,image_id,label,y_pred
0,"[7adbbe8f689e5d5c0593855b056f7019, 8f8d20fc6e9...","[3, 3, 1, 2, 1, 1, 1, 0, 0, 0, 0, 4, 1, 4, 1, ...","[0, 2, 1, 2, 1, 1, 3, 1, 0, 0, 0, 2, 1, 2, 1, ..."
1,"[7adbbe8f689e5d5c0593855b056f7019, 8f8d20fc6e9...","[3, 3, 1, 2, 1, 1, 1, 0, 0, 0, 0, 4, 1, 4, 1, ...","[5, 3, 4, 5, 0, 2, 1, 1, 0, 0, 0, 5, 4, 5, 1, ..."
2,"[7adbbe8f689e5d5c0593855b056f7019, 8f8d20fc6e9...","[3, 3, 1, 2, 1, 1, 1, 0, 0, 0, 0, 4, 1, 4, 1, ...","[3, 1, 1, 2, 1, 1, 5, 1, 1, 1, 0, 5, 1, 1, 1, ..."
3,"[7adbbe8f689e5d5c0593855b056f7019, 8f8d20fc6e9...","[3, 3, 1, 2, 1, 1, 1, 0, 0, 0, 0, 4, 1, 4, 1, ...","[3, 3, 1, 3, 4, 2, 0, 2, 0, 0, 0, 5, 1, 5, 2, ..."
4,"[7adbbe8f689e5d5c0593855b056f7019, 8f8d20fc6e9...","[3, 3, 1, 2, 1, 1, 1, 0, 0, 0, 0, 4, 1, 4, 1, ...","[4, 1, 1, 1, 0, 1, 4, 0, 0, 0, 0, 3, 1, 1, 1, ..."


In [65]:
# 3) Get weighted average
def weight_determ(weights_dict, fold, score):
        return weights_dict[fold][score]
    
weight_determ(weights_dic, 4, 3)


0.08333333333333333

In [67]:
zero = sample_test_set['fold_0_y_pred']
one = sample_test_set['fold_1_y_pred']
two = sample_test_set['fold_2_y_pred']
three = sample_test_set['fold_3_y_pred']
four = sample_test_set['fold_4_y_pred']

zero_w = [weights_dic[0][int(x)] for x in zero]
one_w = [weights_dic[1][int(x)] for x in one]
two_w = [weights_dic[2][int(x)] for x in two]
three_w = [weights_dic[3][int(x)] for x in three]
four_w = [weights_dic[4][int(x)] for x in four]

calc_df = pd.DataFrame()
calc_df['zero']= zero
calc_df['one']= one
calc_df['two']= two
calc_df['three']= three
calc_df['four']= four

calc_df['zero_w']= zero_w
calc_df['one_w']= one_w
calc_df['two_w']= two_w
calc_df['three_w']= three_w
calc_df['four_w']= four_w


'''
sample_test_set['fold_w_avg'] = [(weight_determ(weights_dict, 0, int(x))*x + weight_determ(weights_dict, 1, int(y))*y \
                                 + weight_determ(weights_dict, 2, int(z))*z +weight_determ(weights_dict, 3, int(a))*a \
                                 + weight_determ(weights_dict, 4, int(b))*b) \
                                 / (weight_determ(weights_dict, 0, int(x)) + weight_determ(weights_dict, 1, int(y)) \
                                 + weight_determ(weights_dict, 2, int(z)) + weight_determ(weights_dict, 3, int(a)) \
                                 + weight_determ(weights_dict, 4, int(b)))
                                 for x in zero for y in one for z in two for a in three for b in four]
'''

"\nsample_test_set['fold_w_avg'] = [(weight_determ(weights_dict, 0, int(x))*x + weight_determ(weights_dict, 1, int(y))*y                                  + weight_determ(weights_dict, 2, int(z))*z +weight_determ(weights_dict, 3, int(a))*a                                  + weight_determ(weights_dict, 4, int(b))*b)                                  / (weight_determ(weights_dict, 0, int(x)) + weight_determ(weights_dict, 1, int(y))                                  + weight_determ(weights_dict, 2, int(z)) + weight_determ(weights_dict, 3, int(a))                                  + weight_determ(weights_dict, 4, int(b)))\n                                 for x in zero for y in one for z in two for a in three for b in four]\n"

In [68]:
calc_df['numerator'] = calc_df['zero_w']*calc_df['zero'] + calc_df['one_w']*calc_df['one'] + calc_df['two_w']*calc_df['two'] \
+ calc_df['three_w']*calc_df['three'] + calc_df['four_w']*calc_df['four']

calc_df['denominator'] = calc_df['zero_w']+calc_df['one_w']+calc_df['two_w']+calc_df['three_w']+calc_df['four_w']

calc_df['w_score_float'] = calc_df['numerator']/calc_df['denominator']
calc_df['w_score_int'] = [round(x) for x in calc_df['w_score_float']]
sample_test_set['w_score_int'] = calc_df['w_score_int']

In [69]:
equality = sample_test_set[sample_test_set['isup_grade']==sample_test_set['w_score_int']]
accuracy = len(equality)/len(sample_test_set)
accuracy

0.11